In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad
from torch.autograd import Variable

import numpy as np
import os
import time

import utils

import networks

import matplotlib.pyplot as plt
from tqdm import tqdm, trange

import importlib

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
seed = 1234
torch.set_default_dtype(torch.float32)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [3]:
K = 40
r = 0.05
sigma = 0.25
T = 1
S_range = [0, 130]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)

# Build Neural Network

In [4]:
class Net(nn.Module):
    def __init__(self, layers):
        super(Net, self).__init__()
        self.layers = layers
        self.iter = 0
        self.activation = nn.Tanh()
        self.linear = nn.ModuleList(
            [nn.Linear(layers[i], layers[i + 1]) for i in range(len(layers) - 1)])
        for i in range(len(layers) - 1):
            nn.init.xavier_normal_(self.linear[i].weight.data, gain=1.0)
            nn.init.zeros_(self.linear[i].bias.data)

    def forward(self, x):
        if not torch.is_tensor(x):
            x = torch.from_numpy(x).to(device)
        a = self.activation(self.linear[0](x))
        for i in range(1, len(self.layers) - 2):
            z = self.linear[i](a)
            a = self.activation(z)
        a = self.linear[-1](a)
        return a

layers = [2, 20, 20, 20, 20, 1] # Network structure
net = Net(layers).to(device) #  Network initialization

In [5]:
n_epochs = 60000
lossFunction = nn.MSELoss()

from torchimize.functions import lsq_lma
# coeffs_list = lsq_lma(fnn.parameters(), function=lossFunction)
optimizer = optim.Adam(net.parameters(), lr=0.002)

x_f_s = torch.tensor(0.).float().to(device).requires_grad_(True)
x_label_s = torch.tensor(0.).float().to(device).requires_grad_(True)
optimizer_adam_weight = torch.optim.Adam([x_f_s] + [x_label_s], lr=0.0001)


samples = {"pde": 50000, "bc":5000, "fc":5000}

# Modelling


In [6]:
loss_hist = []
start_time = time.time()

for epoch in range(n_epochs):
    bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    # save training data points to tensor and send to device
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)


    # PDE Round
    y1_hat = net(n_st_train)
    grads = tgrad.grad(y1_hat, n_st_train, grad_outputs=torch.ones(y1_hat.shape).cuda(), retain_graph=True, create_graph=True, only_inputs=True)[0]
    # print(grads)
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    # print(grads2nd)
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*y1_hat)
    
    
    # conditions Round
    y21_hat = net(bc_st_train)
    bc_loss = lossFunction(bc_v_train, y21_hat)
    
    
    # Backpropagation and Update
    optimizer.zero_grad()
    total_loss = pde_loss + bc_loss
    combined_loss = torch.exp(-x_f_s.detach()) * pde_loss + torch.exp(-x_label_s.detach()) * bc_loss
    combined_loss.backward()
    optimizer.step()
    
    loss_hist.append(total_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{n_epochs} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.5f}, nn_loss loss: {combined_loss.item():5f}, total loss: {total_loss.item():.5f}, minimum loss: {min(loss_hist):.5f}')
        print(f'the weight is {torch.exp(-x_f_s.detach()).item():.5f}, {torch.exp(-x_label_s.detach()).item():.5f}')
    
    
    optimizer_adam_weight.zero_grad()
    loss = torch.exp(-x_f_s) * pde_loss.detach() + x_f_s \
            + torch.exp(-x_label_s) * bc_loss.detach() + x_label_s
    loss.backward()
    optimizer_adam_weight.step()
    pass

end_time = time.time()

print('run time:', end_time - start_time)
print('Adam done!')

0/60000 PDE Loss: 0.00611, BC Loss: 3385.67578, nn_loss loss: 3385.681885, total loss: 3385.68188, minimum loss: 3385.68188
the weight is 1.00000, 1.00000
500/60000 PDE Loss: 1.25222, BC Loss: 1667.91895, nn_loss loss: 1599.542725, total loss: 1669.17114, minimum loss: 1669.17114
the weight is 1.00029, 0.95825
1000/60000 PDE Loss: 1.64804, BC Loss: 896.74414, nn_loss loss: 834.867371, total loss: 898.39221, minimum loss: 898.39221
the weight is 0.96094, 0.92923
1500/60000 PDE Loss: 3.49599, BC Loss: 452.60828, nn_loss loss: 414.869507, total loss: 456.10428, minimum loss: 456.10428
the weight is 0.87391, 0.90987
2000/60000 PDE Loss: 4.92314, BC Loss: 212.35120, nn_loss loss: 194.560547, total loss: 217.27434, minimum loss: 217.27434
the weight is 0.79739, 0.89773
2500/60000 PDE Loss: 3.97350, BC Loss: 93.67564, nn_loss loss: 86.376869, total loss: 97.64914, minimum loss: 97.64909
the weight is 0.74097, 0.89065
3000/60000 PDE Loss: 9.45865, BC Loss: 42.29462, nn_loss loss: 44.072441, to